In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from heapq import nsmallest
from sklearn.metrics import pairwise_distances
import random
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [2]:
articles = pd.read_csv('dataset/article_data_newyork', delimiter=',', header=None)
tables = pd.read_csv('dataset/table_data_newyork', delimiter=',', header=None)

In [3]:
formattedArticles = articles.iloc[:,:].values
formattedTables = tables.iloc[:,:].values

In [4]:
embedding_model = Doc2Vec.load('pre_trained_models/doc2vec.bin')

/home/lss9/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
tablesByCharacteristics = []

for tables in tqdm(formattedTables):
    
    table_title = str(tables[1])
    table_page_content = str(tables[2])
    
    catch = table_title+" "+table_page_content
    
    vector_words = tknzr.tokenize(table_title)
    
    word_embedding = embedding_model.infer_vector(vector_words)
    
    tablesByCharacteristics.append(word_embedding)

100%|██████████| 52426/52426 [01:35<00:00, 549.23it/s] 


In [12]:
corpusArticlesID = []
articlesByCharacteristics = []

for articles in tqdm(formattedArticles):
    
    article_url = articles[0]
    article_page_title = articles[1]
    article_main_passage = articles[2][:6000]    
    table_url = articles[3]
    
    catch = article_page_title +" "+article_main_passage
    
    corpusArticlesID.append(table_url)
        
    #embedding
    vector_words = tknzr.tokenize(article_main_passage)
    
    word_embedding = embedding_model.infer_vector(vector_words)
    
    articlesByCharacteristics.append(word_embedding)

100%|██████████| 148/148 [00:08<00:00, 18.04it/s]


In [7]:
def getIdRankedTalbes(topK,distanceVector):

    idRankedTables = []

    for topkDistance in topK:
        
        index = np.where(distanceVector == topkDistance)
         
        indexColummun = index[0][0]
        
        idRankedTables.append(formattedTables[indexColummun][0])

    return idRankedTables

In [8]:
def getAccuracy(idRankedTables, idQueryGoal):

    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [9]:
def saveAccuracy(k,accuracy):
    
    if k == 1:
            
        AverageTop1.append(accuracy)
    
    if k == 5:
            
        AverageTop5.append(accuracy)
        
    if k == 10:
            
        AverageTop10.append(accuracy)
        
    if k == 100:
            
        AverageTop100.append(accuracy)
        
    if k == 1000:
            
        AverageTop1000.append(accuracy)

In [16]:
AverageTop1 = []
AverageTop5 = []
AverageTop10 = []
AverageTop100 = []
AverageTop1000 = []

topK = [1,5,10,100,1000]

for i in tqdm(range(len(articlesByCharacteristics))):
    
    distanceVector = pairwise_distances(articlesByCharacteristics[i].reshape(1,300), tablesByCharacteristics, metric='cosine')
    
    idQueryGoal = corpusArticlesID[i]
    
    for accuracyK in topK:
        
        countTopTables = accuracyK
        
        topKRank = nsmallest(countTopTables, distanceVector[0])
    
        idRankedTables = getIdRankedTalbes(topKRank,distanceVector[0])
        
        accuracy_value = getAccuracy(idRankedTables,idQueryGoal)
        
        #save the accuracy on the list
        saveAccuracy(accuracyK,accuracy_value)

100%|██████████| 148/148 [04:32<00:00,  1.01s/it]


In [17]:
print(str(round(np.mean(AverageTop1),4)))
print(str(round(np.mean(AverageTop5),4)))
print(str(round(np.mean(AverageTop10),4)))
print(str(round(np.mean(AverageTop100),4)))
print(str(round(np.mean(AverageTop1000),4)))

0.1689
0.3919
0.473
0.7838
0.9527
